In [170]:
from sentence_transformers import SentenceTransformer
import numpy as np
import json
import pandas as pd
from tqdm.notebook import tqdm

In [171]:
model = SentenceTransformer(
    "all-MiniLM-L6-v2"
) 
df = pd.read_json("hotel_concatenated_descriptoin_sentiment_final.json", orient='records')
#df = pd.read_json("./startups_demo.json", lines=True)




In [172]:
vectors = model.encode(
    [row.hotel_description for row in df.itertuples()],
    show_progress_bar=True,
)

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

In [173]:
np.save("hotel_vectors.npy", vectors, allow_pickle=False)



In [174]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

qdrant_client = QdrantClient("http://localhost:6333")

In [175]:
qdrant_client.recreate_collection(
    collection_name="hotel_descriptions",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

True

In [176]:
with open("./hotel_concatenated_descriptoin_sentiment_final.json", "r") as fd:
    # Load JSON data with records orientation
    data = json.load(fd)

# Define payload as an iterator over the startup data
payload = iter(data)

In [177]:

# Load all vectors into memory, numpy array works as iterable for itself.
# Other option would be to use Mmap, if you don't want to load all data into RAM
vectors = np.load("./hotels_vectors.npy")

In [178]:
qdrant_client.upload_collection(
    collection_name="hotel_descriptions",
    vectors=vectors,
    payload=payload,
    ids=None,  # Vector ids will be assigned automatically
    batch_size=256,  # How many vectors will be uploaded in a single request?
)